In [ ]:
function update_as!(as, ϕs, rs)
    # iterate unique k-values, frequencies
    @inbounds @fastmath for i ∈ 1:3
        
        # kr * s_factor
        re_i = ϕs[2,i] * rs[i]
        im_i = ϕs[3,i] * rs[i]
        
        for j ∈ 1:3
            idx1 = 3(i-1) + j
            idx2 = idx1 + 9
        
            # ωt
            re_j = ϕs[2,j+3]
            im_j = ϕs[3,j+3]
        
            a = re_i * re_j - im_i * im_j
            b = re_i * im_j + im_i * re_j

            as.re[idx1] = a
            as.re[idx2] = a
            as.im[idx1] = b
            as.im[idx2] = b
            
        end
        
    end
    return nothing
end
@btime update_as!($as, $ϕs, $rs)

In [ ]:
# 18 -> 6
idxs_kr = Int32.([
    1 1 1 1 1 1 2 2 2 2 2 2 3 3 3 3 3 3
])
idxs_ωt = Int32.([
    4 4 4 4 4 4 5 5 5 5 5 5 6 6 6 6 6 6
])

@inline function update_as!(idxs_kr, idxs_ωt, as, ϕs, rs)
    # iterate unique k-values, frequencies
    @inbounds @fastmath for i ∈ eachindex(idxs_kr)
        
        idx_kr = idxs_kr[i]
        ϕs_kr_re = ϕs[2,idx_kr]
        ϕs_kr_im = ϕs[3,idx_kr]
        r = rs[idx_kr]
        
        # kr * s_factor
        re1 = ϕs_kr_re * r
        im1 = ϕs_kr_im * r
        
        idx_ωt = idxs_ωt[i]
        ϕs_ωt_re = ϕs[2,idx_ωt]
        ϕs_ωt_im = ϕs[3,idx_ωt]
        r = rs[idx_kr]

        # ωt
        re2 = ϕs_ωt_re
        im2 = ϕs_ωt_im

        a = re1 * re2 - im1 * im2
        b = re1 * im2 + im1 * re2

        as.re[i] = a
        as.re[i] = a
        as.im[i] = b
        as.im[i] = b
        
    end
    return nothing
end
@btime update_as!($idxs_kr, $idxs_ωt, $as, $ϕs, $rs)

In [ ]:
ϕs_kr = MMatrix{2,3}(zeros(T,2,3))
ϕs_ωt = MMatrix{2,3}(zeros(T,2,3))
@inline function update_ϕs_individual!(ϕs_kr, ϕs_ωt, ϕs)
    @turbo for i ∈ axes(ϕs_kr,2)
        ϕs_kr[1,i] = ϕs[2,i]
        ϕs_kr[2,i] = ϕs[3,i]
    end
    @turbo for i ∈ axes(ϕs_ωt,2)
        ϕs_ωt[1,i] = ϕs[2,i+3]
        ϕs_ωt[2,i] = ϕs[3,i+3]
    end
    return nothing
end
@btime update_ϕs_individual!($ϕs_kr, $ϕs_ωt, $ϕs)

In [ ]:
@inline function update_Hge_k!(Hge_k, d_ge, kEs)
    for k ∈ 1:3
        E = view(kEs, k, :)
        Hge = view(Hge_k, :, :, k)
        update_Hge!(Hge, d_ge, E)
    end
    return nothing
end
@btime update_Hge_k!($(p_fast.Hge_k), $(p_fast.d_ge), $(p_fast.kEs))

In [ ]:
@inline function update_Hge_k!(Hge_k, d_ge, kEs)
    for k ∈ 1:3
        @turbo for i ∈ axes(Hge_k,1)
            for j ∈ axes(Hge_k,2)
                Hge_ij_re = zero(eltype(Hge))
                Hge_ij_im = zero(eltype(Hge))
                for q ∈ 1:3
                    E_kq_re = kEs.re[k,q]
                    E_kq_im = kEs.im[k,q]
                    a = E_kq_re * d_ge[i,j,q]
                    b = E_kq_im * d_ge[i,j,q]
                    Hge_ij_re -= a # minus sign to make sure Hamiltonian is -d⋅E
                    Hge_ij_im -= b
                end
                Hge_k.re[i,j,k] = Hge_ij_re
                Hge_k.im[i,j,k] = Hge_ij_im
            end
        end
    end
    return nothing
end
@btime update_Hge!($(p_fast.Hge_k), $(p_fast.d_ge), $(p_fast.kEs))

In [1]:
@inline function update_Hge!(Hge, d_ge, E)
    @turbo for i ∈ axes(Hge,1)
        for j ∈ axes(Hge,2)
            Hge_ij_re = zero(eltype(Hge))
            Hge_ij_im = zero(eltype(Hge))
            for q ∈ 1:3
                E_q_re = E.re[q]
                E_q_im = E.im[q]
                a = E_q_re * d_ge[i,j,q]
                b = E_q_im * d_ge[i,j,q]
                Hge_ij_re -= a # minus sign to make sure Hamiltonian is -d⋅E
                Hge_ij_im -= b
            end
            Hge.re[i,j] = Hge_ij_re
            Hge.im[i,j] = Hge_ij_im
        end
    end
    return nothing
end
@btime update_Hge!($(p_fast.Hge), $d_ge, $(p_fast.E_total))

SyntaxError: invalid character '∈' (U+2208) (1507748477.py, line 2)

In [ ]:
@inline function update_Hge!(Hge, Hge_k)
    @turbo for i ∈ axes(Hge,1)
        for j ∈ axes(Hge,2)
            Hge_re = zero(eltype(Hge.re))
            Hge_im = zero(eltype(Hge.re))
            for k ∈ 1:3
                Hge_re += Hge_k.re[i,j,k]
                Hge_im += Hge_k.im[i,j,k]
            end
            Hge.re[i,j] = Hge_re
            Hge.im[i,j] = Hge_im
        end
    end
    return nothing
end
@btime update_Hge!($(p_fast.Hge), $(p_fast.Hge_k))

In [ ]:
"""
    Evalute ψ_q = d_q ψ.

    Break up psi into ground and excited?
"""
@inline function evaluate_ψq(ψ_q, d, ψ)
    @turbo for i ∈ axes(d,1)
        ψ_re_i = ψ.re[i]
        ψ_im_i = ψ.im[i]
        for j ∈ axes(d,2)
            ψ_re_j = ψ.re[j+12]
            ψ_im_j = ψ.im[j+12]
            for q ∈ 1:3

                d_q_ij = d[i,j,q]
                ψq_re = d_q_ij * ψ_re_i
                ψq_im = d_q_ij * ψ_im_i

                ψ_q.re[i,q] -= d_q_ij * ψ_re_j
                ψ_q.im[i,q] -= d_q_ij * ψ_im_j
                ψ_q.re[j+12,q] -= d_q_ij * ψ_re_i
                ψ_q.im[j+12,q] -= d_q_ij * ψ_im_i

            end
        end
    end
end
@btime evaluate_ψq($(p_fast.ψ_q), $(p_fast.d_ge), $(p_fast.ψ0))

In [ ]:
@inline function update_Hge!(Hge, d_ge, E)
    @turbo for i ∈ axes(Hge,1)
        for j ∈ axes(Hge,2)
            Hge_ij_re = zero(eltype(Hge))
            Hge_ij_im = zero(eltype(Hge))
            for q ∈ 1:3
                E_q_re = E.re[q]
                E_q_im = E.im[q]
                a = E_q_re * d_ge[i,j,q]
                b = E_q_im * d_ge[i,j,q]
                Hge_ij_re -= a # minus sign to make sure Hamiltonian is -d⋅E
                Hge_ij_im -= b
            end
            Hge.re[i,j] = Hge_ij_re
            Hge.im[i,j] = Hge_ij_im
        end
    end
    return nothing
end
@btime update_Hge!($(p_fast.Hge), $(p_fast.d_ge), $(p_fast.E_total))

In [ ]:
"""
    Evalute ψ_q = d_q ψ.

    Break up psi into ground and excited?
"""
@inline function evaluate_ψq(ψ_q, d, ψ)
    for j ∈ axes(d,2)
        # ψ_re_j = ψ.re[j]
        # ψ_im_j = ψ.im[j]
        for i ∈ axes(d,1)
            # ψ_re_i = ψ.re[i]
            # ψ_im_i = ψ.im[i]
            for q ∈ 1:3
                d_q_ij = d[i,j,q]
                ψq_re = d_q_ij * ψ_re_i
                ψq_im = d_q_ij * ψ_im_i
                ψ_q.re[i,q] -= d_q_ij * ψ_re_j
                ψ_q.im[i,q] -= d_q_ij * ψ_im_j
                # ψ_q.re[j,q] -= d_q_ij * ψ_re_i
                # ψ_q.im[j,q] -= d_q_ij * ψ_im_i
            end
        end
    end
end
@btime evaluate_ψq($(p_fast.ψ_q), $(p_fast.d_ge), $(p_fast.ψ0))

In [2]:
@inline function update_ψq!(ψ_q, ψ, ds, ds_state1, ds_state2)
    @inbounds @fastmath for q ∈ 1:3
        ds_state1_q = ds_state1[q]
        ds_state2_q = ds_state2[q]
        ds_q = ds[q]
        
        for i ∈ eachindex(ds_state1_q)
            m = ds_state1_q[i]
            n = ds_state2_q[i]
            d = ds_q[i]
            
            ψ_e_re = ψ.re[m]
            ψ_e_im = ψ.im[m]
            ψ_g_re = ψ.re[n]
            ψ_g_im = ψ.im[n]
            
            ψ_q.re[n,q] = d * ψ_e_re
            ψ_q.im[n,q] = d * ψ_e_im
            ψ_q.re[m,q] = d * ψ_g_re
            ψ_q.im[m,q] = d * ψ_g_im
        end
    end
end
@btime update_ψq!($(p_fast.ψ_q), $(p_fast.ψ0), $(p_fast.ds), $(p_fast.ds_state1), $(p_fast.ds_state2))

SyntaxError: invalid character '∈' (U+2208) (1028654345.py, line 2)